In [ ]:
install.packages('e1071')
install.packages('rgdal')
install.packages('randomForest')
install.packages('rpart')
install.packages('neuralnet')


In [ ]:
install.packages('e1071')

In [1]:
library(rgdal)
library(randomForest)
library(rpart)
library(neuralnet)
library(e1071)


Warning message:
"package 'rgdal' was built under R version 3.6.3"
Loading required package: sp

Warning message:
"package 'sp' was built under R version 3.6.3"
rgdal: version: 1.5-23, (SVN revision 1121)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.2.1, released 2020/12/29
Path to GDAL shared files: C:/Users/LeiWang/anaconda3/envs/r_env/Lib/R/library/rgdal/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 7.2.1, January 1st, 2021, [PJ_VERSION: 721]
Path to PROJ shared files: C:/Users/LeiWang/anaconda3/envs/r_env/Lib/R/library/rgdal/proj
PROJ CDN enabled: FALSE
Linking to sp version:1.4-5
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rgdal_show_exportToProj4_warnings"="none") before loading rgdal.
Overwritten PROJ_LIB was C:/Users/LeiWang/anaconda3/envs/r_env/Lib/R/library/rgdal/proj

Warning message:
"package 'randomForest' was built under R version 3.6.3"
randomForest 4.6-14

Type

In [2]:
sourcename<-"usace_multisource.csv"# S2
#sourcename<-"usace_compare.csv"#s3
#sourcename<-"usace_compare_greaterthan2days.csv" #S4,5
#sourcename<-"USACE_lake_training.csv" #S1
#sourcename<-"harsha_laks_20150921_water_quality.csv"
water_data <- read.csv(file = sourcename)
n = dim(water_data)[1]
set.seed(100)
train_index = sample(1:n, 0.75*n)
test_index = setdiff(1:n, train_index)

train_data = water_data[train_index,]
test_data = water_data[test_index,]


In [3]:
water_data

Lat,Lon,Chl_a,id,B2,B3,B4,B5,B6,B7
36.87481,-86.06550,12.30,9,226.0000,323.0000,171.0000,112.0000,50.0000,26.0000
36.89778,-86.09972,12.30,10,214.0000,261.0000,145.0000,103.0000,32.0000,14.0000
36.84417,-86.07139,15.10,12,242.0000,323.0000,206.0000,173.0000,131.0000,92.0000
36.91444,-86.05639,18.50,13,214.0000,300.0000,167.0000,103.0000,35.0000,17.0000
36.92714,-86.04458,21.50,14,223.0000,334.0000,184.0000,109.0000,34.0000,20.0000
36.81139,-86.07000,18.40,15,219.0000,316.0000,172.0000,117.0000,35.0000,19.0000
38.44530,-85.28140,19.23,0,227.0000,385.0000,256.0000,311.0000,133.0000,80.0000
38.00283,-85.30567,7.40,81,266.0000,341.0000,203.0000,194.0000,72.0000,45.0000
39.60278,-84.98889,7.90,19,580.0000,909.0000,909.0000,479.0000,154.0000,103.0000
39.44088,-85.00274,16.60,20,306.0000,364.0000,229.0000,296.0000,174.0000,81.0000


In [4]:
f<-as.formula("Chl_a~B2+B3+B4+B5+B6+B7")
#f<-as.formula("Chl_a~FLAASH__B1+FLAASH__B2+FLAASH__B3+FLAASH__B4+FLAASH__B5+FLAASH_B6+FLAASH_B7")
#f<-as.formula("Chl_a~B2_1+B3_1+B4_1+B5_1+B6_1+B7_1")
#install.packages("GA")
 tryCatch(library(GA),
         error = function(e){
           message("An error occurred:\n", e)
           install.packages('GA')
             
         })
library(GA)
#fine-tuning using the GA algorithm
K = 5 # 5-fold cross-validation
fold_inds <- sample(1:K, nrow(water_data), replace = TRUE)
lst_CV_data <- lapply(1:K, function(i) list(
    train_data = water_data[fold_inds != i, , drop = FALSE], 
    test_data = water_data[fold_inds == i, , drop = FALSE]))

# Given the values of parameters 'cost', 'gamma' and 'epsilon', return the rmse of the model over the test data
evalParams <- function(train_data, test_data, cost, gamma, epsilon) {
    # Train
    model <- svm(f, data = train_data, cost = cost, gamma = gamma)
    # Test
    rmse <- mean((predict(model, newdata = test_data) - test_data$Chl_a) ^ 2)
    return (rmse)
}

# Fitness function (to be maximized)
# Parameter vector x is: (cost, gamma, epsilon)
fitnessFunc <- function(x, Lst_CV_Data) {
    # Retrieve the SVM parameters
    cost_val <- x[1]
    gamma_val <- x[2]

  

    # Use cross-validation to estimate the RMSE for each split of the dataset
    rmse_vals <- sapply(Lst_CV_Data, function(in_data) with(in_data, 
        evalParams(train_data, test_data, cost_val, gamma_val)))

    # As fitness measure, return minus the average rmse (over the cross-validation folds), 
    # so that by maximizing fitness we are minimizing the rmse
    return (-mean(rmse_vals))
}

# Range of the parameter values to be tested
# Parameters are: (cost, gamma, epsilon)
theta_min <- c(cost = 1e-3, gamma = 1e-2)
theta_max <- c(cost = 10, gamma = 2)

# Run the genetic algorithm
results <- ga(type = "real-valued", fitness = fitnessFunc, lst_CV_data, 
    names = names(theta_min), 
    lower = theta_min, upper = theta_max,
    popSize = 100, maxiter = 50)

summary(results)

Warning message:
"package 'GA' was built under R version 3.6.3"
Loading required package: foreach

Loading required package: iterators

Package 'GA' version 3.2.1
Type 'citation("GA")' for citing this R package in publications.


Attaching package: 'GA'


The following object is masked from 'package:utils':

    de




-- Genetic Algorithm ------------------- 

GA settings: 
Type                  =  real-valued 
Population size       =  100 
Number of generations =  50 
Elitism               =  5 
Crossover probability =  0.8 
Mutation probability  =  0.1 
Search domain = 
       cost gamma
lower 1e-03  0.01
upper 1e+01  2.00

GA results: 
Iterations             = 50 
Fitness function value = -38.27502 
Solution = 
         cost    gamma
[1,] 5.226312 1.845826

In [4]:
 tryCatch(library(MLmetrics),
         error = function(e){
           message("An error occurred:\n", e)
           install.packages('MLmetrics')
             
         })
library(MLmetrics)
#f<-as.formula("Chl__ug_L_~LC08_b1+LC08_b2+LC08_b3+LC08_b4+LC08_b5+LC08_b6+LC08_b7")
#f<-as.formula("Chl__ug_L_~FLAASH__B1+FLAASH__B2+FLAASH__B3+FLAASH__B4+FLAASH__B5+FLAASH_B6+FLAASH_B7")
#f<-as.formula("Chl_a~B2+B3+B4+B5+B6+B7")
#f<-as.formula("Chl_a~B2_1+B3_1+B4_1+B5_1+B6_1+B7_1")
svr<-svm(f,train_data, cost = 5.226312          , gamma = 1.845826 )
train_data$pred<-predict(svr,train_data)
test_data$pred <-predict(svr,test_data)
error_train<-train_data$Chl_a - train_data$pred
error_test<-test_data$Chl_a  - test_data$pred
rmse <- function(error)
{
  sqrt(mean(error^2))
}
TrainRMSE <- rmse(error_train) 
TrainRMSE
TestRMSE <- rmse(error_test)
TestRMSE
mape <- MAPE(test_data$pred,test_data$Chl_a)
mape

Warning message:
"package 'MLmetrics' was built under R version 3.6.3"

Attaching package: 'MLmetrics'


The following object is masked from 'package:base':

    Recall




ERROR: Error in svm(f, train_data, cost = 5.226312, gamma = 1.845826): object 'f' not found


In [ ]:
library(ggplot2)
plot(test_data$pred,test_data$Chl_a)

In [ ]:
#predict the EPA Chl-a values
f<-as.formula("Chl_a~B2+B3+B4+B5+B6+B7")
svr<-svm(f,water_data, cost = 8.979  , gamma = 1.995, , elipson =  0.03014179) 
sourcename<-"greenriverpnt.csv"
#sourcename<-"USACE_lake_training.csv"
HAB_data <- read.csv(file = sourcename)

HAB_data$pred <-predict(svr,HAB_data)


In [ ]:
write.csv(HAB_data, file = 'Greenriverpnt_pred.csv')